# Noise Corrector CNN to get CR images without noise
---

---

In [1]:
# This is just a function to allow toggleing code cells that are too long for good
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide_toggle()

In [2]:
import torch #should be installed by default in any colab notebook
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import json
import os
import pandas as pd
import h5py
from time import time
from datetime import datetime
from IPython import display as display_IPython

assert torch.cuda.is_available(), "GPU is not enabled"

# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the functions and routines for the DL
### Define the model and its constructor

In [3]:
class Proximity_Metric_Based_On_Simple_Encoder(nn.Module):
    def __init__(self, X=302, feats_1=15, feats_2=20, feats_3=20, feats_4=20,
                 prop1=3, prop2=2, prop3=1, av_pool1_div=4, conv4_feat_size=15, av_pool2_div=10, 
                 out_fc_1=10, out_fc_2=10,
                 dropout_p1=0.2, dropout_p2=0.1
                ): 
        # propj is such that the_ image getting out from stage j is propj/prop_{j-1}-ths of the previous (with j=0 being 5)
        # clearly, prop_{j-1}>prop_{j}>...
        # 2X+1 will be assumed to be divisible by 5
        assert((2*X+1)%5==0)
        assert(prop1>prop2)
        assert(prop2>prop3)
        assert((int((prop3*(2*X+1)/5)/av_pool1_div)-conv4_feat_size)>0)
        
        
        super(Proximity_Metric_Based_On_Simple_Encoder, self).__init__()
        # in is [epoch_size, 1, 2X+1, 2X+1]
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=feats_1, 
                               kernel_size = int((2*X+1)/5*(5-prop1)+1), bias=True) 
        # out conv1 [epoch_size, feats_1, prop1*(2X+1)/5, prop1*(2X+1)/5]
        self.conv2 = nn.Conv2d(in_channels=feats_1, out_channels=feats_2, 
                               kernel_size = int((2*X+1)/5*(prop1-prop2)+1), bias=True) 
        # out conv1 [epoch_size, feats_2, prop2*(prop1*(2X+1)/5)/prop1, prop2*(prop1*(2X+1)/5)/prop1]
        # that is [epoch_size, feats_2, prop2*(2X+1)/5), prop2*(2X+1)/5)]
        self.conv3 = nn.Conv2d(in_channels=feats_2, out_channels=feats_3, 
                               kernel_size = int((2*X+1)/5*(prop2-prop3)+1), bias=True)
        # out conv3 is [epoch_size, feats_3, prop3*(2X+1)/5), prop3*(2X+1)/5)]

        self.avPool1 = nn.AvgPool2d(kernel_size= int((prop3*(2*X+1)/5)*(1-1/av_pool1_div)) +1, stride=1)
        # out avpool1 is [epoch_size, feats_3, prop3*(2X+1)/5)/av_pool1_div, prop3*(2X+1)/5)/av_pool1_div]
        
        self.conv4 = nn.Conv2d(in_channels=feats_3, out_channels=feats_4, 
                              kernel_size= int((prop3*(2*X+1)/5)/av_pool1_div+1)-conv4_feat_size+1, bias=True)
        # [epoch_size, feats_4, conv4_feat_size, conv4_feat_size]
        
        self.avPool2 = nn.AvgPool2d(kernel_size= int(conv4_feat_size*(1-1/av_pool2_div)) +1, stride=1)
        # out avpool1 is [epoch_size, feats_4, conv4_feat_size/av_pool2_div+1, conv4_feat_size/av_pool2_div+1]
        
        #self.in_fc = int(feats_4*(conv4_feat_size/av_pool2_div+1)**2)
        self.in_fc = feats_4*((((((2*X+1-int((2*X+1)/5*(5-prop1)+1)+1)
                                  -int((2*X+1)/5*(prop1-prop2)+1)+1)
                                 -int((2*X+1)/5*(prop2-prop3)+1)+1)
                                -int((prop3*(2*X+1)/5)*(1-1/av_pool1_div)) -1+1)
                               -int((prop3*(2*X+1)/5)/av_pool1_div+1)+conv4_feat_size-1+1)
                              -int(conv4_feat_size*(1-1/av_pool2_div)) -1+1)**2
        
        self.fc1 = nn.Linear(in_features=self.in_fc, out_features=out_fc_1, bias=True)
        self.fc2 = nn.Linear(in_features=out_fc_1, out_features=out_fc_2, bias=True)
        
        self.dropout1 = nn.Dropout(p=dropout_p1, inplace=False)
        self.dropout2 = nn.Dropout(p=dropout_p2, inplace=False)
        self.relu = torch.nn.functional.leaky_relu

        self.batchNorm2 = nn.BatchNorm2d(num_features=feats_2)
        self.batchNorm4 = nn.BatchNorm2d(num_features=feats_4)

    def forward(self, x): # [batch_size, 2X+1, 2X+1] or [batch_size, 1, 2X+1, 2X+1]
        x = x.view(x.shape[0], 1, x.shape[-2], x.shape[-1]).float() # [batch_size, 1, 2X+1, 2X+1]
        # Normalize to unity the float image
        x = x/x.amax(dim=(2,3), keepdim=True)[0] # [batch_size, 1, 2X+1, 2X+1]
        
        x = self.relu( self.conv1(x) ) # [batch_size, feats_1, prop1*(2X+1)/5, prop1*(2X+1)/5]
        
        x = self.batchNorm2( self.relu( self.conv2(self.dropout1(x)) )) # [batch_size, feats_2, prop2*(2X+1)/5, prop2*(2X+1)/5]

        
        x = self.relu( self.conv3(self.dropout2(x)) ) # [batch_size, feats_3, prop3*(2X+1)/5, prop3*(2X+1)/5]

        
        x = self.avPool1(x) # [batch_size, feats_3, prop3*(2X+1)/5)/av_pool1_div, prop3*(2X+1)/5)/av_pool1_div]

        
        x = self.batchNorm4(self.conv4(self.dropout2(x))) # [batch_size, feats_4, conv4_feat_size, conv4_feat_size]

        
        x = self.relu( self.avPool2(x) ) # [batch_size, feats_4, conv4_feat_size/av_pool2_div, conv4_feat_size/av_pool2_div]

        
        x = x.view(x.shape[0], self.in_fc) #[batch_size, feats_4*int(conv4_feat_size/av_pool2_div)**2]

        
        x = self.fc2( self.relu( self.fc1(x) ) ) #[batch_size, out_fc_2]
        
        return x
    
    def print_shapes(self, batch_size=10, X=302):
        x = torch.ones((batch_size, 1, 2*X+1, 2*X+1)).to(device)
        print(f"Initial shape {x.shape}")
        x = self.relu( self.conv1(x) ) # [batch_size, feats_1, prop1*(2X+1)/5, prop1*(2X+1)/5]
        print(f"Post Conv1+relu shape {x.shape}")
        x = self.batchNorm2( self.relu( self.conv2(self.dropout1(x)) )) # [batch_size, feats_2, prop2*(2X+1)/5, prop2*(2X+1)/5]
        print(f"Post drop1+Conv2+relu+batchnorm shape {x.shape}")
        
        x = self.relu( self.conv3(self.dropout2(x)) ) # [batch_size, feats_3, prop3*(2X+1)/5, prop3*(2X+1)/5]
        print(f"Post drop2+Conv3+relu shape {x.shape}")
        
        x = self.avPool1(x) # [batch_size, feats_3, prop3*(2X+1)/5)/av_pool1_div, prop3*(2X+1)/5)/av_pool1_div]
        print(f"Post Av Pool1 shape {x.shape}")
        
        x = self.batchNorm4(self.conv4(self.dropout2(x))) # [batch_size, feats_4, conv4_feat_size, conv4_feat_size]
        print(f"Post drop2+Conv4+batchnorm shape {x.shape}")
        
        x = self.relu( self.avPool2(x) ) # [batch_size, feats_4, conv4_feat_size/av_pool2_div, conv4_feat_size/av_pool2_div]
        print(f"Post Av. Pool2 shape {x.shape}")
        
        x = x.view(x.shape[0], self.in_fc) #[batch_size, feats_4*int(conv4_feat_size/av_pool2_div)**2]
        print(f"Post Pre-fc shape {x.shape}")
        
        x = self.fc2( self.relu( self.fc1(x) ) ) #[batch_size, out_fc2]
        print(f"Post fc1+relu+fc2 shape {x.shape}")

    




class Proximity_Metric_Based_On_Corrector(nn.Module):
    def __init__(self, S0=2*302+1, S1=2*290+1, S2=2*250+1, S3=2*200+1, S4 = 2*10+1,
                 S5 = 2*1+1, S6 =2,
                 feats_S1=10, feats_S2=10, feats_S3=20, feats_S4=20, feats_S5 = 20,
                 out_fc1=100, out_fc2=10,
                 feats_S6 = 25,
                 dropout_p=0.1
                ): 
       
        super(Proximity_Metric_Based_On_Corrector, self).__init__()
        self.Ss = [S0, S1, S2, S3, S4, S5, S6]
        self.feats = [1, feats_S1, feats_S2, feats_S3, feats_S4, feats_S5, feats_S6]
        self.out_fc1 = out_fc1
        self.out_fc2 = out_fc2
        # in is [batch_size, 1, S0, S0]
        self.conv_S01 = nn.Conv2d(in_channels=1, out_channels=feats_S1, 
                               kernel_size = S0-S1+1, bias=True) 
        # out conv_S01 [batch_size, feats_S1, S1, S1]
        self.conv_S12 = nn.Conv2d(in_channels=feats_S1, out_channels=feats_S2, 
                               kernel_size = S1-S2+1, bias=True) 
        # out conv_S12 [batch_size, feats_S2, S2, S2]
        self.conv_S23 = nn.Conv2d(in_channels=feats_S2, out_channels=feats_S3, 
                               kernel_size = S2-S3+1, bias=True) 
        # out conv_S23 [batch_size, feats_S3, S3, S3]
        
        self.conv_S33 = nn.Conv2d(in_channels=feats_S3, out_channels=feats_S3, 
                               kernel_size = 1, bias=True) 
        # out conv_S33 [batch_size, feats_S3, S3, S3]
        
        self.conv_S34 = nn.Conv2d(in_channels=feats_S3, out_channels=feats_S4, 
                               kernel_size = S3-S4+1, bias=True) 
        # out conv_S34 [batch_size, feats_S4, S4, S4]
        
        self.conv_S45 = nn.Conv2d(in_channels=feats_S4, out_channels=feats_S5, 
                               kernel_size = S4-S5+1, bias=True) 
        # out conv_S45 [batch_size, feats_S5, S5, S5]
        self.conv_S56 = nn.Conv2d(in_channels=feats_S5, out_channels=feats_S6, 
                               kernel_size = S5-S6+1, bias=True) 
        # out conv_S56 [batch_size, feats_S6, S6, S6]
        
        self.in_fc1 = S6*S6*feats_S6
        self.fc1 = nn.Linear(in_features=self.in_fc1, out_features=out_fc1, bias=True)
        self.fc2 = nn.Linear(in_features=out_fc1, out_features=out_fc2, bias=True)
        
        self.dropout = nn.Dropout(p=dropout_p, inplace=False)
        self.relu = torch.nn.functional.leaky_relu

        self.batchNorm1 = nn.BatchNorm2d(num_features=feats_S3)
        self.batchNorm2 = nn.BatchNorm1d(num_features=out_fc1)
        

    def forward(self, x): # [batch_size, 2X+1, 2X+1] or [batch_size, 1, 2X+1, 2X+1]
        x = x.view(x.shape[0], 1, x.shape[-2], x.shape[-1]).float() # [batch_size, 1, 2X+1, 2X+1]
        # Normalize to unity the float image
        x = x/x.amax(dim=(2,3), keepdim=True)[0] # [batch_size, 1, 2X+1, 2X+1]
        
        # Conv layers
        x = self.relu(self.conv_S01(x)) # [batch_size, feats_S1, S1, S1]
        x = self.dropout( self.relu(self.conv_S12(x)) ) # [batch_size, feats_S2, S2, S2]
        x = self.relu(self.conv_S23(x)) # [batch_size, feats_S3, S3, S3]
        x = self.batchNorm1(self.relu(self.conv_S33(x)))
        x = self.relu(self.conv_S34(x))
        x = self.relu(self.conv_S45(x))
        x = self.relu(self.conv_S56(x))
        
        x = x.view(x.shape[0], self.in_fc1)
        x = self.dropout( self.relu(self.batchNorm2(self.fc1(self.dropout(x)))) )
        return self.fc2(x)

        
    def print_shapes(self, batch_size=10):
        x = torch.ones((batch_size, 1, self.Ss[0], self.Ss[0])).to(device)
        print(f"Initial shape {x.shape}")
        x = self.relu(self.conv_S01(x)) # [batch_size, feats_S1, S1, S1]
        print(f"Conv01 shape {x.shape} should be [{batch_size},{self.feats[1]},{self.Ss[1]}, {self.Ss[1]}]")
        x = self.dropout( self.relu(self.conv_S12(x)) ) # [batch_size, feats_S2, S2, S2]
        print(f"Conv12 shape {x.shape} should be [{batch_size},{self.feats[2]},{self.Ss[2]}, {self.Ss[2]}]")
        x = self.batchNorm1( self.relu(self.conv_S23(x)) ) # [batch_size, feats_S3, S3, S3]
        print(f"Conv23 shape {x.shape} should be [{batch_size},{self.feats[3]},{self.Ss[3]}, {self.Ss[3]}]")
        x = self.relu(self.conv_S34(x))  # [batch_size, feats_S3, S3, S3]
        print(f"Conv34 shape {x.shape} should be [{batch_size},{self.feats[4]},{self.Ss[4]}, {self.Ss[4]}]")
        x = self.relu(self.conv_S45(x))  # [batch_size, feats_S3, S3, S3]
        print(f"Conv45 shape {x.shape} should be [{batch_size},{self.feats[5]},{self.Ss[5]}, {self.Ss[5]}]")
        x = self.relu(self.conv_S56(x))  # [batch_size, feats_S3, S3, S3]
        print(f"Conv56 shape {x.shape} should be [{batch_size},{self.feats[6]},{self.Ss[6]}, {self.Ss[6]}]")
        x = x.view(x.shape[0], self.in_fc1)
        print(f"Input to fc1 is {x.shape} should be [{batch_size}, {self.in_fc1}]")
        x = self.dropout( self.relu(self.batchNorm2(self.fc1(self.dropout(x)))) )
        print(f"Output of fc1 is {x.shape} should be [{batch_size}, {self.out_fc1}]")
        x= self.fc2(x)
        print(f"Output of fc2 is {x.shape} should be [{batch_size}, {self.out_fc2}]")
        

In [4]:
hide_toggle(for_next=True)


In [4]:
# subroutine to count number of parameters in the model
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.numel()
    return np

### The routines to validate and train

In [6]:
hide_toggle(for_next=True)


In [5]:
@torch.no_grad()  # prevent this function from computing gradients 
def validate_epoch(criterion, model, dataloader, per_epoch_use_max_batches=None): #show_confusion_matrix = False):
    if per_epoch_use_max_batches is None:
        per_epoch_use_max_batches = len(dataloader)
    val_loss = 0
    max_abs_error = torch.Tensor([0]).to(device)
    mean_abs_error = 0
    preds = torch.Tensor().to(device)
    targets = torch.Tensor().to(device)

    model.eval() # disable the dropout, among others

    for batch_id in range(len(dataloader)):  
        data, target = dataloader[batch_id] # dataloader sends them to device already   
        prediction = model(data)
        target = target.view(prediction.shape)
        loss = criterion(prediction, target)
        val_loss += loss.item()             
        max_abs_error = torch.maximum(torch.max(torch.abs(prediction-target), 0).values, max_abs_error)
        mean_abs_error += torch.sum(torch.mean(torch.abs(prediction-target), (-2,-1)), 0)
        if batch_id % per_epoch_use_max_batches == per_epoch_use_max_batches-1:
            break
    val_loss /= min(len(dataloader), per_epoch_use_max_batches)
    mean_abs_error /= min(len(dataloader), per_epoch_use_max_batches)
    #accuracy = 100. * correct / len(loader.dataset)
    print(f'\nValidation set: Average loss: {val_loss:.4f}, Average Abs Error: {np.array(mean_abs_error.cpu())}, Maximum Abs Error: {np.array(max_abs_error.cpu())} \n')

    #if show_confusion_matrix:
    #    visualize_confusion_matrix(preds.to(torch.device('cpu')), targets.to(torch.device('cpu')))

    return val_loss


def train_epoch(epoch, criterion, model, optimizer, dataloader, print_loss_every_batches=20,
                optimizer_step_every_batches=1, per_epoch_use_max_batches=None):
    if per_epoch_use_max_batches is None:
        per_epoch_use_max_batches = len(dataloader)
        
    total_loss = 0.0

    model.train()

    optimizer.zero_grad()
    #t = time()
    random_indices = np.random.choice(range(min(len(dataloader), per_epoch_use_max_batches)), len(dataloader), replace=False)
    for batch_id, idx in enumerate(random_indices):  
        data, target = dataloader[idx] # dataloader sends them to device already
        #data, target = data.to(device), target.to(device)
        prediction = model(data) # data is [batch_size, embedding_dim]
        loss = criterion(prediction, target)
        loss.backward()
        
        if batch_id % optimizer_step_every_batches==optimizer_step_every_batches-1:
            optimizer.step()
            optimizer.zero_grad()
        # print loss every N batches
        if batch_id % print_loss_every_batches == print_loss_every_batches-1:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_id+1) * len(data), len(dataloader)*dataloader.batch_size,
                100*(batch_id+1)*len(data) / (len(dataloader)*dataloader.batch_size), loss.item()))

        #if batch_id % per_epoch_use_max_batches == per_epoch_use_max_batches-1:
        #    break


        total_loss += loss.item()  #.item() is very important here
        # In order to avoid having total_loss as a tensor in the gpu
        #t = time()

    return total_loss / min(len(dataloader), per_epoch_use_max_batches)

### The full training loop

In [8]:
hide_toggle(for_next=True)


In [6]:
def full_training_loop(model, criterion, optimizer, train_loader, test_loader, epochs=10,
                       print_loss_every_batches=20, validate_every_epochs=2, optimizer_step_every_batches=1,
                      per_epoch_use_max_train_batches=None, per_epoch_use_max_test_batches=None,
                      image_path=None, save_model_every_epochs=1, model_path=None, best_model_path=None):
    losses = {"train": [], "val": []}
    %matplotlib inline
    for epoch in range(epochs):

        train_loss = train_epoch(epoch, criterion, model, optimizer, train_loader,
                                 print_loss_every_batches=print_loss_every_batches,
                                optimizer_step_every_batches=optimizer_step_every_batches,
                                per_epoch_use_max_batches=per_epoch_use_max_train_batches)
        if epoch%validate_every_epochs==0 and epoch!=0:
            val_loss = validate_epoch(criterion, model, test_loader, per_epoch_use_max_test_batches)
        else:
            try:
                val_loss = losses["val"][-1]
            except:
                val_loss = train_loss
        if epoch and train_loss<=min(losses["train"]) and best_model_path:
            torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            }, best_model_path)
        
        losses["train"].append(train_loss)
        losses["val"].append(val_loss)        
        plt.plot(losses["train"], label="training loss")
        plt.plot(losses["val"], label="validation loss")
        #plt.yscale('log')
        plt.legend()
        if image_path is not None:
            plt.savefig(image_path)
            plt.clf()
        else:
            display_IPython.clear_output(wait=True)
            plt.pause(0.001)
            plt.show()
        if epoch % save_model_every_epochs==save_model_every_epochs-1 and model_path:
            torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            }, model_path)

    return losses

### The Dataset class and Data Sampler

In [10]:
hide_toggle(for_next=True)

In [7]:
from torch.utils.data import Dataset
from torchvision.io import read_image
from torch.utils.data import DataLoader

class ImageDataloader(Dataset):
    def __init__(self, GT_file_path, h5f_full_path, P, K, produce_batches_per_epoch):
        self.df_GTs = pd.DataFrame.from_dict(json.load(open(GT_file_path))) 
        # Que te linkee el nombre de la entrada del h5f (un index del batch) con los ground truth phis

        self.h5f = h5py.File(f"{h5f_full_path}", 'r')
        self.num_diff_perfects = len(self.h5f)
        shape = self.h5f[ self.df_GTs[0]['ID'] ].shape
        self.max_K = shape[0]
        self.image_size = shape[1:]
        print(f"There are {self.num_diff_perfs} total different classes (w0,R0,phiCR) triplets.")
        print(f"For each of them, there are the denoised and {self.max_K-1} noisy versions")
        print(f"Each image is of shape {self.image_size}")
        print(f"A total of {self.num_diff_perfs*self.max_K} different images in the dataset, with their GT phiCRs")
        self.K = min(K, self.max_K) # number of versions of each class to employ per batch
        self.P = P # number of different "classes" to employ per batch
        self.produce_batches_per_epoch = produce_batches_per_epoch
        
    def __len__(self):
        return self.produce_batches_per_epoch

    def __getitem__(self, idx):
        # sudar olimplicamente del idx y simplemente devolver una combinación que tenga que ser aleatoria
        random_indices = np.random.choice(range(self.num_diff_perfs), self.P, replace=False)
        data = torch.zeros((self.P, self.K, self.image_size[0], self.image_size[1]), 
                          device=device, dtype=torch.float32) # [P,K,2X+1,2X+1]
        #gt_phiCR = torch.zeros((self.P, 1), device=device, dtype=torch.float32) #[P,1]
        for ind in random_indices:
            data[i,:,:,:] = torch.tensor( np.array(self.h5f[str(ind)]), sevice=device, dtype=torch.float32 ) 
            #gt_phiCR[i] = self.df_GTs['ID'==str(i)]['phiCR']
        return data #, gt_phiCR
    # solo devuelve las imagenes ya subidas a la gpu, luego fuera hay que hacer el cómputo de la métrica
    # y hacer el pairwise todo!
    
def my_loss(output, target):
    loss = torch.mean((output - target)**2)
    return loss

In [28]:
b=np.array([1,2,3])
c=np.array([3,4,7])
np.linalg.norm(c-2*b)

1.4142135623730951

In [46]:
a = torch.tensor([[1,2,3],[3,8,3],[3,4,7], [2,4,6]], dtype=torch.float32)
d = torch.pdist(a, p=2)

In [47]:
d

tensor([6.3246, 4.8990, 3.7417, 5.6569, 5.0990, 1.4142])

In [59]:
m = torch.zeros((4, 4), device=device)

triu_indices = torch.triu_indices(row=4, col=4, offset=1)
print(triu_indices)
m[triu_indices[0], triu_indices[1]] = d.to(device)

tensor([[0, 0, 0, 1, 1, 2],
        [1, 2, 3, 2, 3, 3]])


In [61]:
m.T[triu_indices[0], triu_indices[1]]=d.to(device)
m

tensor([[0.0000, 6.3246, 4.8990, 3.7417],
        [6.3246, 0.0000, 5.6569, 5.0990],
        [4.8990, 5.6569, 0.0000, 1.4142],
        [3.7417, 5.0990, 1.4142, 0.0000]], device='cuda:0')

---
# Initialize the dataset and sampler (choose the number of batches per epoch, and their length) and fix the artificial noise hyperparameters

Note that since in each epoch the dataset shown to the model will be random, we can use the same dataset as a validation set.

In [8]:
dataset_path = "/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/Noisy_Non_Noisy_same_angle/TRAIN/"
GT_file_path_train = f"{dataset_path}/TRAIN/GROUND_TRUTHS_K=4_Noisy_Non_Noisy_same_angle.json"
images_h5_path_train = f"{dataset_path}/TRAIN/Dataset_K=4_Noisy_Non_Noisy_same_angle.h5" 
GT_file_path_test = f"{dataset_path}/TRAIN/GROUND_TRUTHS_K=4_Noisy_Non_Noisy_same_angle.json"
images_h5_path_test = f"{dataset_path}/TRAIN/Dataset_K=4_Noisy_Non_Noisy_same_angle.h5" 

save_stuff_path = f"/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/SIMULATIONS/Proximity_Metric/"

total_epochs = 10000
validate_every_epochs = 11000
optimizer_step_every_batches = 3
per_epoch_use_max_train_batches= 100
per_epoch_use_max_test_batches=20
save_model_every_epochs = 1
torch.manual_seed(678)

K=4
P=10

#exp_name='Proximity_Metric_from_Image_Corrector'
exp_name='Proximity_Metric_from_Simple_Encoder'


In [16]:
training_data = ImageDataloader(GT_file_path_train, images_h5_path_train, P, K, per_epoch_use_max_train_batches)
test_data = ImageDataloader(GT_file_path_test, images_h5_path_test, P, K, per_epoch_use_max_test_batches)

FileNotFoundError: [Errno 2] No such file or directory: '/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/Noisy_Non_Noisy_same_angle/TRAIN//TRAIN/GROUND_TRUTHS_K=4_Noisy_Non_Noisy_same_angle.json'

# Fix the Hyperparameters and Initialize the Model and the Optimizer

In [15]:
# DENOISER BASED METRIC ############
X=302
S0=2*X+1
S1=2*250+1
S2=2*200+1
S3=2*150+1
S4=2*10+1
S5=2*1+1
S6=2
feats_S1=5
feats_S2=5
feats_S3=10
feats_S4=20
feats_S5=20
feats_S6=25
out_fc1=100
out_fc2=100
dropout_p=0.1

# SIMPLE ENCODER TO phiCR BASED METRIC ###########
X=302
feats_1=20
feats_2=20
feats_3=20
feats_4=5
prop1=2.5
prop2=1.5
prop3=0.6
av_pool1_div=2
conv4_feat_size=8
av_pool2_div=10
out_fc_1=5
dropout_p1=0.2
dropout_p2=0.1

In [16]:
model = Proximity_Metric_Based_On_Corrector(S0=S0, S1=S1, S2=S2, S3=S3, S4=S4, S5=S5, S6=S6,
                 feats_S1=feats_S1, feats_S2=feats_S2, feats_S3=feats_S3, feats_S4=feats_S4,
                 feats_S5=feats_S5, feats_S6=feats_S6,
                 out_fc1=out_fc1, out_fc2=out_fc2,
                 dropout_p=dropout_p ) 

'''
model = Proximity_Metric_Based_On_Simple_Encoder( X=X, feats_1=feats_1, feats_2=feats_2, feats_3=feats_3, feats_4=feats_4,
                 prop1=prop1, prop2=prop2, prop3=prop3, av_pool1_div=av_pool1_div, conv4_feat_size=conv4_feat_size, 
                av_pool2_div=av_pool2_div, 
                 out_fc_1=out_fc_1,
                 dropout_p1=dropout_p1, dropout_p2=dropout_p2 ) 
'''

print(f"Number of parameters {get_n_params(model)}")

# In case we wish to transfer the learned parameters of another run
#check_file="Noisy_Model_and_Optimizer_2022-01-18 11:37:26.485463.pt"
#checkpoint = torch.load(save_stuff_path+f"/NNs/{check_file}")

# move model to gpu if available
model.to(device)
model.print_shapes(batch_size=3)

#model.load_state_dict(checkpoint['model'])


# Initialize the weights of the model! Default initialization might already be fine!

# we can use a MSE loss for the regression task we have in hands
criterion = nn.MSELoss()

# we will choose as optimizer the 
#optimizer = torch.optim.Adagrad(model.parameters(), lr=0.1, lr_decay=0.01, weight_decay=0.3,
#                                initial_accumulator_value=0, eps=1e-10)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
#optimizer.load_state_dict(checkpoint['optimizer'])

Number of parameters 16779415
Initial shape torch.Size([3, 1, 605, 605])
Conv01 shape torch.Size([3, 5, 501, 501]) should be [3,5,501, 501]
Conv12 shape torch.Size([3, 5, 401, 401]) should be [3,5,401, 401]
Conv23 shape torch.Size([3, 10, 301, 301]) should be [3,10,301, 301]
Conv34 shape torch.Size([3, 20, 21, 21]) should be [3,20,21, 21]
Conv45 shape torch.Size([3, 20, 3, 3]) should be [3,20,3, 3]
Conv56 shape torch.Size([3, 25, 2, 2]) should be [3,25,2, 2]
Input to fc1 is torch.Size([3, 100]) should be [3, 100]
Output of fc1 is torch.Size([3, 100]) should be [3, 100]
Output of fc2 is torch.Size([3, 100]) should be [3, 100]


# Run the Training

In [ ]:
%%time
losses = full_training_loop(model, criterion, optimizer, training_data, training_data, 
                    epochs=total_epochs, print_loss_every_batches=2100,
                            validate_every_epochs=validate_every_epochs,
                           optimizer_step_every_batches=optimizer_step_every_batches,
                           per_epoch_use_max_train_batches=per_epoch_use_max_train_batches, 
                            per_epoch_use_max_test_batches=per_epoch_use_max_test_batches,
                           image_path=save_stuff_path+ f"/Training_Loss_{datetime.now()}_{exp_name}.png",
                           save_model_every_epochs=save_model_every_epochs, 
                            model_path=save_stuff_path+f"/Model_and_Optimizer_{datetime.now()}_{exp_name}.pt",
                            best_model_path=save_stuff_path+f"/BEST_Model_and_Optimizer_{datetime.now()}_{exp_name}.pt"
                           )
# Execute the training and validation

# Save the resulting model weights

In [ ]:
torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            }, save_stuff_path+f"FINAL_Model_and_Optimizer_{datetime.now()}_{exp_name}.pt")

# Final Validation

In [ ]:
print("\n\n\nFINAL VALIDATION! ####################################################\n\n")
print("Train Set")
#validate_epoch(nn.MSELoss(), model, sampler, dataset, per_epoch_use_max_train_batches)
print("Test Set")
validate_epoch(nn.MSELoss(), model, test_dataloader, per_epoch_use_max_test_batches)

In [3]:
hide_toggle(True)

In [ ]:
def compute_intensity_gravity_centers(images):
    """
        Expects input image to be an array of dimensions [N_imgs, h, w].
        It will return an array of gravity centers [N_imgs, 2(h,w)] in pixel coordinates
        Remember that pixel coordinates are set equal to array indices

    """
    # image wise total intensity and marginalized inensities for weighted sum
    intensity_in_w = torch.sum(images, dim=1) # weights for x [N_images, raw_width]
    intensity_in_h = torch.sum(images, dim=2) # weights for y [N_images, raw_height]
    total_intensity = intensity_in_h.sum(dim=1) # [N_images]

    # Compute mass center for intensity
    # [N_images, 2] (h_center,w_center)
    return torch.nan_to_num( torch.stack(
        (torch.matmul(intensity_in_h.float(), torch.arange(images.shape[1], 
                                    dtype=torch.float32, device=device))/total_intensity,
         torch.matmul(intensity_in_w.float(), torch.arange(images.shape[2], 
                                    dtype=torch.float32, device=device))/total_intensity),
        dim=1
        ), nan=0.0, posinf=None, neginf=None)

def compute_raw_to_centered_iX(images, X=302):

        g_raw = compute_intensity_gravity_centers(images) # [ N_images, 2]

        # crop the iamges with size (X+1+X)^2 leaving the gravity center in
        # the central pixel of the image. In case the image is not big enough for the cropping,
        # a 0 padding will be made.
        centered_images = torch.zeros( ( images.shape[0], 2*X+1, 2*X+1),  dtype = images.dtype, 
                                      device=device)

        # we round the gravity centers to the nearest pixel indices
        g_index_raw = torch.round(g_raw).int() #[ N_images, 2]

        # obtain the slicing indices around the center of gravity
        # TODO -> make all this with a single array operation by stacking the lower and upper in
        # a new axis!!
        # [ N_images, 2 (h,w)]
        unclipped_lower = g_index_raw-X
        unclipped_upper = g_index_raw+X+1

        # unclipped could get out of bounds for the indices, so we clip them
        lower_bound = torch.clip( unclipped_lower.float(), min=torch.Tensor([[0,0]]).to(device),
                                 max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(device)).int()
        upper_bound = torch.clip( unclipped_upper.float(), min=torch.Tensor([[0,0]]).to(device),
                                 max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(device)).int()
        # we use the difference between the clipped and unclipped to get the necessary padding
        # such that the center of gravity is left still in the center of the image
        padding_lower = lower_bound-unclipped_lower
        padding_upper = upper_bound-unclipped_upper

        # crop the image
        for im in range(g_raw.shape[0]):
            centered_images[im, padding_lower[ im, 0]:padding_upper[ im, 0] or None,
                                        padding_lower[ im, 1]:padding_upper[ im, 1] or None] = \
                      images[im, lower_bound[ im, 0]:upper_bound[ im, 0],
                                          lower_bound[ im, 1]:upper_bound[ im, 1]]

        return centered_images

In [4]:
import os
import cv2
plot3d_resolution=0.3

path = "/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/SIMULATIONS/UMAP_Regressor/TEST_IMAGES/"
image_names = os.listdir(f"{path}")

predicted_corrections={}
problem_images = {}
os.makedirs(f"{save_stuff_path}/Test_Corrections/", exist_ok=True)

for im_n in image_names:
    model.eval()
    im = cv2.imread(path+im_n, cv2.IMREAD_ANYDEPTH)
    im_type = im.dtype
    max_int = 2**8-1 if im_type==np.uint8 else 2**16-1 if im_type==np.uint16 else 2**32-1 if im_type==np.uint32 else 2**53-1 if im_type==np.uint64 else None
    im = compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))
    corr_im = model(im)[0] #[1, 2X+1, 2X+1]
    print(corr_im.shape)
    corr_im = (self.max_intensity*(corr_im/corr_im.amax(dim=(1,2), keepdim=True)[0].unsqueeze(1)))
    corr_im = np.asarray(corr_im.to('cpu').squeeze(0)).astype(im_type)
    print(corr_im.shape)
    predicted_corrections[im_n] = corr_im
    im = np.asarray(im.to('cpu').squeeze(0)).stype(im_type)
    problem_images[im_n] = im
    
    # On the one hand we want to plot the resulting corrections on the test set
    fig = plt.figure(figsize=(2*4.5, 2*4.5))
    axes=fig.subplots(2,2)

    cm=axes[0, 0].imshow(im, cmap='viridis')
    cm2 = axes[0,1].imshow(corr_im, cmap='viridis')
    axes[0,0].grid(True)
    axes[0,1].grid(True)

    axes[1,0].set_visible(False)
    axes[1,1].set_visible(False)
    ax = fig.add_subplot(223, projection='3d')
    Xs,Ys = np.meshgrid(np.arange(im.shape[0]),np.arange(im.shape[1]))
    fig.suptitle(f"Intesity Profiles for Image\n{im_n}")
    cbax=fig.add_axes([0.54,0.05,0.4,0.01])
    fig.colorbar(cm, ax=axes[0,0], cax=cbax, orientation='horizontal')
    theta=25
    phi=30
    ax.plot_surface(Xs, Ys, im.T, rcount=int(len(prof_y)*plot3d_resolution), ccount=int(len(prof_x)*plot3d_resolution), cmap='viridis') # rstride=1, cstride=1, linewidth=0
    ax.set_xlabel('Y')
    #ax.set_xlim(-8, 8)
    ax.set_ylabel('X')
    #ax.set_ylim(-10, 8)
    ax.set_zlabel('Intensity')
    ax.set_zlim(-0.078*np.max(im), np.max(im))
    ax.set_title("Image intensity 3D plot")
    ax.view_init(10, theta)
    #ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1.3, 1.3, 1.3, 1]))

    ax = fig.add_subplot(224, projection='3d')
    Xs,Ys = np.meshgrid(np.arange(corr_im.shape[0]),np.arange(corr_im.shape[1]))
    fig.suptitle(f"Intesity Profiles for Image\n{im_n}")
    theta=25
    phi=30
    ax.plot_surface(Xs, Ys, corr_im.T, rcount=int(corr_im.shape[0]*plot3d_resolution), ccount=int(corr_im.shape[1]*plot3d_resolution), cmap='viridis') # rstride=1, cstride=1, linewidth=0
    ax.set_xlabel('Y')
    #ax.set_xlim(-8, 8)
    ax.set_ylabel('X')
    #ax.set_ylim(-10, 8)
    ax.set_zlabel('Intensity')
    ax.set_zlim(-0.078*np.max(corr_im), np.max(corr_im))
    ax.set_title("Image intensity 3D plot")
    ax.view_init(10, theta)
    #ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1.3, 1.3, 1.3, 1]))
    fig.savefig(f"{save_stuff_path}/Test_Corrections/{im_n}")
    
    

# We now save them with the correct directory structure such that we can use the Todor Algorithms on them
# This will work as a benchmark of how well the correction was made
for i in ["Problem", "Reference"]:
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/17_18/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/18_19/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/28_29/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/43_44/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/70_71/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/con_los_dos/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/non_noisy_5_6/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/non_noisy_72_73/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/ortog/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/ref_vs_ref/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/sin_el_negativo/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/sin_el_positivo/{i}", exist_ok=True)
    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/17_18/Problem/Corr_17.png", predicted_corrections['IM_53017_phiCR_0.659442126750946.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/17_18/Reference/Corr_18.png", predicted_corrections['IM_53018_phiCR_-2.2813968658447266.png'])    

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/18_19/Problem/Corr_18.png", predicted_corrections['IM_53018_phiCR_-2.2813968658447266.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/18_19/Reference/Corr_19.png", predicted_corrections['IM_53019_phiCR_-2.679948091506958.png'])    

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/28_29/Problem/Corr_28.png", predicted_corrections['IM_52928_phiCR_0.6789670586585999.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/28_29/Reference/Corr_29.png", predicted_corrections['IM_52929_phiCR_0.9714600443840027.png'])    

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/43_44/Problem/Corr_43.png", predicted_corrections['IM_43_phiCR_-1.57120680809021.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/43_44/Reference/Corr_44.png", predicted_corrections['IM_44_phiCR_2.6544740200042725.png'])  

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/70_71/Problem/Corr_70.png", predicted_corrections['IM_40870_phiCR_-0.6731816530227661.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/70_71/Reference/Corr_71.png", predicted_corrections['IM_40871_phiCR_-2.4470927715301514.png'])  

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/con_los_dos/Problem/Corr_con_los_dos.png", predicted_corrections['con_los_dos.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/con_los_dos/Reference/Corr_ref1.png", predicted_corrections['sin_los_dos_solo_tubo.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/con_los_dos/Reference/Corr_ref2.png", predicted_corrections['antes_de_la_estandar.png'])  


cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_negativo/Problem/Corr_sin_el_negativo.png", predicted_corrections['sin_el_negativo.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_negativo/Reference/Corr_ref1.png", predicted_corrections['sin_los_dos_solo_tubo.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_negativo/Reference/Corr_ref2.png", predicted_corrections['antes_de_la_estandar.png'])  


cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_positivo/Problem/Corr_sin_el_positivo.png", predicted_corrections['sin_el_positivo.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_positivo/Reference/Corr_ref1.png", predicted_corrections['sin_los_dos_solo_tubo.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_positivo/Reference/Corr_ref2.png", predicted_corrections['antes_de_la_estandar.png'])  

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/ref_vs_ref/Problem/Corr_ref1.png", predicted_corrections['sin_los_dos_solo_tubo.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/ref_vs_ref/Reference/Corr_ref2.png", predicted_corrections['antes_de_la_estandar.png'])  


cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/ortog/Problem/Corr_90.png", predicted_corrections['90__100.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/ortog/Reference/Corr_Ref_90.png", predicted_corrections['Reference__100.png'])  

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/non_noisy_5_6/Problem/Corr_5.png", predicted_corrections['IM_5_phiCR_-2.6049387454986572.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/non_noisy_5_6/Reference/Corr_6.png", predicted_corrections['IM_6_phiCR_-1.7562638521194458.png'])  

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/non_noisy_72_73/Problem/Corr_72.png", predicted_corrections['IM_72_phiCR_-2.946422576904297.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/non_noisy_72_73/Reference/Corr_73.png", predicted_corrections['IM_73_phiCR_1.33404541015625.png'])  


os.system(f"python ../../ANALYSIS_SCRIPTS/CODE_Get_Angle_Live.py {save_path_stuff+"/Todor_Benchmark/"} {1} False")

SyntaxError: invalid syntax (<ipython-input-4-62f00fefed67>, line 17)

In [ ]:
raise ValueError

# Charge models and do inference

In [ ]:
checkpoint = torch.load(f"/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/SIMULATIONS/Simple_Encoder/Noisy_Model_and_Optimizer_2022-01-24 19:57:31.886991.pt")

model = Simple_Encoder( X=X, feats_1=feats_1, feats_2=feats_2, feats_3=feats_3, feats_4=feats_4,
                 prop1=prop1, prop2=prop2, prop3=prop3, av_pool1_div=av_pool1_div, conv4_feat_size=conv4_feat_size, av_pool2_div=av_pool2_div, 
                 out_fc_1=out_fc_1,
                 dropout_p1=dropout_p1, dropout_p2=dropout_p2 ) 

model.to(device)
model.load_state_dict(checkpoint['model'])

In [ ]:
def compute_intensity_gravity_centers(images):
    """
        Expects input image to be an array of dimensions [N_imgs, h, w].
        It will return an array of gravity centers [N_imgs, 2(h,w)] in pixel coordinates
        Remember that pixel coordinates are set equal to array indices

    """
    # image wise total intensity and marginalized inensities for weighted sum
    intensity_in_w = torch.sum(images, dim=1) # weights for x [N_images, raw_width]
    intensity_in_h = torch.sum(images, dim=2) # weights for y [N_images, raw_height]
    total_intensity = intensity_in_h.sum(dim=1) # [N_images]

    # Compute mass center for intensity
    # [N_images, 2] (h_center,w_center)
    return torch.nan_to_num( torch.stack(
        (torch.matmul(intensity_in_h.float(), torch.arange(images.shape[1], 
                                    dtype=torch.float32, device=device))/total_intensity,
         torch.matmul(intensity_in_w.float(), torch.arange(images.shape[2], 
                                    dtype=torch.float32, device=device))/total_intensity),
        dim=1
        ), nan=0.0, posinf=None, neginf=None)

def compute_raw_to_centered_iX(images, X=302):

        g_raw = compute_intensity_gravity_centers(images) # [ N_images, 2]

        # crop the iamges with size (X+1+X)^2 leaving the gravity center in
        # the central pixel of the image. In case the image is not big enough for the cropping,
        # a 0 padding will be made.
        centered_images = torch.zeros( ( images.shape[0], 2*X+1, 2*X+1),  dtype = images.dtype, 
                                      device=device)

        # we round the gravity centers to the nearest pixel indices
        g_index_raw = torch.round(g_raw).int() #[ N_images, 2]

        # obtain the slicing indices around the center of gravity
        # TODO -> make all this with a single array operation by stacking the lower and upper in
        # a new axis!!
        # [ N_images, 2 (h,w)]
        unclipped_lower = g_index_raw-X
        unclipped_upper = g_index_raw+X+1

        # unclipped could get out of bounds for the indices, so we clip them
        lower_bound = torch.clip( unclipped_lower.float(), min=torch.Tensor([[0,0]]).to(device),
                                 max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(device)).int()
        upper_bound = torch.clip( unclipped_upper.float(), min=torch.Tensor([[0,0]]).to(device),
                                 max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(device)).int()
        # we use the difference between the clipped and unclipped to get the necessary padding
        # such that the center of gravity is left still in the center of the image
        padding_lower = lower_bound-unclipped_lower
        padding_upper = upper_bound-unclipped_upper

        # crop the image
        for im in range(g_raw.shape[0]):
            centered_images[im, padding_lower[ im, 0]:padding_upper[ im, 0] or None,
                                        padding_lower[ im, 1]:padding_upper[ im, 1] or None] = \
                      images[im, lower_bound[ im, 0]:upper_bound[ im, 0],
                                          lower_bound[ im, 1]:upper_bound[ im, 1]]

        return centered_images

In [ ]:
# Create and display a FileChooser widget
from ipyfilechooser import FileChooser
path="/home/oiangu/Desktop/Conical_Refraction_Polarimeter"
fc = FileChooser(path+'/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/')
display(fc)

### Choose a single experimental image to predict

In [ ]:
import cv2
image_full_path=fc.selected
#image_full_path="/home/oiangu/Desktop/Conical_Refraction_Polarimeter/Experimental_Stuff/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/sin_el_positivo.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)
if im is None:
    print(f" Unable to import image {image_full_path}")
    raise ValueError
# Center in gravicenter, generating iX
im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

### Plot its Profiles

In [ ]:
plot3d_resolution=0.7

%matplotlib notebook

prof_x=np.sum(im, axis=0)
prof_y=np.sum(im, axis=1)
fig = plt.figure(figsize=(2*4.5, 2*4.5))
axes=fig.subplots(2,2)

cm=axes[0, 0].imshow(im, cmap='viridis')
axes[0,0].grid(True)
axes[0,1].scatter(prof_y, np.arange(len(prof_y)), s=1, label=f'Intensity profile in y')
axes[0,1].set_ylim((0,len(prof_y)))
axes[0,1].invert_yaxis()
axes[1,0].scatter(np.arange(len(prof_x)), prof_x, s=1, label=f'Intensity profile in y')
axes[1,0].set_xlim((0,len(prof_x)))
axes[1,0].invert_yaxis()
axes[0,0].set_xlabel("x (pixels)")
#axes[0,0].set_ylabel("y (pixels)")
axes[0,1].set_xlabel("Cummulative Intensity")
axes[0,1].set_ylabel("y (pixels)")
axes[1,0].set_ylabel("Cummulative Intensity")
axes[1,0].set_xlabel("x (pixels)")
axes[1,0].grid(True)
axes[0,1].grid(True)
axes[1,1].set_visible(False)
ax = fig.add_subplot(224, projection='3d')
Xs,Ys = np.meshgrid(np.arange(len(prof_y)),np.arange(len(prof_x)))
fig.suptitle(f"Intesity Profiles for Image\n{image_full_path.split('/')[-1]}")
files_for_gif=[]
cbax=fig.add_axes([0.54,0.05,0.4,0.01])
fig.colorbar(cm, ax=axes[0,0], cax=cbax, orientation='horizontal')
theta=25
phi=30
ax.plot_surface(Xs, Ys, im.T, rcount=int(len(prof_y)*plot3d_resolution), ccount=int(len(prof_x)*plot3d_resolution), cmap='viridis') # rstride=1, cstride=1, linewidth=0
#cset = ax.contourf(X, Y, im, 2, zdir='z', offset=-20, cmap='viridis', alpha=0.5)
#cset = ax.contourf(X, Y, im, 1, zdir='x', offset=-8, cmap='viridis')
#cset = ax.contourf(X, Y, im, 1, zdir='y', offset=0, cmap='viridis')
ax.set_xlabel('Y')
#ax.set_xlim(-8, 8)
ax.set_ylabel('X')
#ax.set_ylim(-10, 8)
ax.set_zlabel('Intensity')
ax.set_zlim(-0.078*np.max(im), np.max(im))
ax.set_title("Image intensity 3D plot")
ax.view_init(10, theta)
#ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1.3, 1.3, 1.3, 1]))
plt.show()

### Get NN predictions for $R_0, w_0, \phi_{CR}, Z$

In [ ]:
print("Custom")
model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
print(f"Predicted phi_CR {predictions[0]} rad {predictions[0]*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {predictions[0]/2} rad {predictions[0]*180/np.pi/2} deg")

In [ ]:
import cv2
print("Referencia sin nada\n")
%matplotlib inline
image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/sin_los_dos_solo_tubo.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
ref=predictions[0].item()
print(f"Predicted phi_CR {ref} rad {ref*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {ref/2} rad {ref*180/np.pi/2} deg")

In [ ]:
print("Referencia sin nada\n")
%matplotlib inline
image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/antes_de_la_estandar.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
ref2=predictions[0].item()
print(f"Predicted phi_CR {ref2} rad {ref2*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {ref2/2} rad {ref2*180/np.pi/2} deg")

In [ ]:
print("Sin el negativo\n")
%matplotlib inline

image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/sin_el_negativo.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
pos=predictions[0].item()
print(f"Predicted phi_CR {pos} rad {pos*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {pos/2} rad {pos*180/np.pi/2} deg")

In [ ]:
print("Sin el positivo\n")
%matplotlib inline

image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/sin_el_positivo.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
neg = predictions[0].item()
print(f"Predicted phi_CR {neg} rad {neg*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {neg/2} rad {neg*180/np.pi/2} deg")

In [ ]:
print("Con ambos\n")
%matplotlib inline

image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/con_los_dos.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
both=predictions[0].item()
print(f"Predicted phi_CR {both} rad {both*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {both/2} rad {both*180/np.pi/2} deg")


In [ ]:
print("Ref Ort\n")
%matplotlib inline

image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day3/Reference/Reference__100.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
ref_ort=predictions[0].item()
print(f"Predicted phi_CR {ref_ort} rad {ref_ort*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {ref_ort/2} rad {ref_ort*180/np.pi/2} deg")


In [ ]:
print("Ref Ort\n")
%matplotlib inline

image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day3/Problem/90__100.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
ort=predictions[0].item()
print(f"Predicted phi_CR {ref_ort} rad {ort*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {ort/2} rad {ort*180/np.pi/2} deg")


In [ ]:
print(f"Positivo-Ref deberian ser {13.85} deg son {(pos-ref)*180/np.pi/2} deg")
print(f"Negativo-Ref deberian ser {9.45} deg son {(neg-ref)*180/np.pi/2} deg")
print(f"Ambos-Ref deberian ser {4.4} deg son {(both-ref)*180/np.pi/2} deg\n")

print(f"Positivo-Ref2 deberian ser {13.85} deg son {(pos-ref2)*180/np.pi/2} deg")
print(f"Negativo-Ref2 deberian ser {9.45} deg son {(neg-ref2)*180/np.pi/2} deg")
print(f"Ambos-Ref2 deberian ser {4.4} deg son {(both-ref2)*180/np.pi/2} deg\n")

print(f"Ref2-Ref deberian ser {0} deg son {(ref2-ref)*180/np.pi/2} deg\n")

print(f"El de noventa deberian ser {90} deg son {(ref_ort-ort)*180/np.pi/2} deg")

### Get the non-black-box algorithm estimate for $\phi_{CR}$

In [ ]:
import os
os.chdir(f"../../..")
import sys
from SOURCE.CLASS_CODE_GPU_Classes import *
from SOURCE.CLASS_CODE_Image_Manager import *
from SOURCE.CLASS_CODE_Polarization_Obtention_Algorithms import Rotation_Algorithm, Mirror_Flip_Algorithm, Gradient_Algorithm
import numpy as np
import json
import cv2
import pandas as pd
import matplotlib.pyplot as plt

image=im.copy()
saturation=0.9
pol_or_CR="pol" 
deg_or_rad="deg" # for the final output
image_depth=8 # or 16 bit per pixel
image_shortest_side=540
randomization_seed=666
recenter_average_image=False


# 5. POLARIZATION RELATIVE ANGLES ###################################
# Mirror with affine interpolation & Rotation Algorithms will be employed
# Each using both Fibonacci and Quadratic Fit Search
# Also a gradient algorithm
theta_min_Rot=-np.pi
theta_max_Rot=np.pi
rad_min_Grav=3
rad_max_Grav=image_shortest_side
theta_min_Mir=0
theta_max_Mir=np.pi
initial_guess_delta_rad=0.1
initial_guess_delta_pix=10
use_exact_gravicenter=True
precision_quadratic=1e-10
max_it_quadratic=100
cost_tolerance_quadratic=1e-14
precision_fibonacci=1e-10
max_points_fibonacci=100
cost_tolerance_fibonacci=1e-14


##################################################################
##################################################################
im_type=np.uint16 if image_depth==16 else np.uint8
max_intensity=65535 if image_depth==16 else 255
np.random.seed(randomization_seed)
polCR=1 if pol_or_CR=='CR' else 0.5

# 6. POLARIZATION RELATIVE ANGLES ###################################
# Mirror with affine interpolation & Rotation Algorithms will be employed
# Each using both Fibonacci and Quadratic Fit Search
# Results will be gathered in a table and outputed as an excel csv
# Mock Image Loader
# Computar el angulo de cada uno en un dataframe donde una de las entradas sea results y haya un result per fibo qfs y per rotation y mirror affine. Y luego procesar en un 7º paso estos angulos para obtener los angulos relativos etc y perhaps hacer tablucha con ground truth menos el resulting delta angle medido por el algoritmo
image_loader = Image_Manager(mode=X, interpolation_flag=None)
# Define the ROTATION ALGORITHM
rotation_algorithm = Rotation_Algorithm(image_loader,
    theta_min_Rot, theta_max_Rot, None,
    initial_guess_delta_rad, use_exact_gravicenter, initialize_it=False)

# Define the Affine Mirror algorithm
mirror_algorithm = Mirror_Flip_Algorithm(image_loader,
    theta_min_Mir, theta_max_Mir, None,
    initial_guess_delta_rad, method="aff", left_vs_right=True, use_exact_gravicenter=use_exact_gravicenter, initialize_it=False)

# Define the Gradient algorithm
gradient_algorithm = Gradient_Algorithm(image_loader,
        rad_min_Grav, rad_max_Grav,
        initial_guess_delta_pix,
        use_exact_gravicenter)

# A dictionary to gather all the resulting angles for each image

individual_image_results = { 'polarization_method':[], 'optimization_1d':[], 'found_phiCR':[], 'predicted_opt_precision':[] }

def to_result_dict(result_dict, alg, alg_name, opt_name, im_names):
    for key, name in zip(alg.times.keys(), im_names):
        result_dict['polarization_method'].append(alg_name)
        result_dict['optimization_1d'].append(opt_name)
        result_dict['found_phiCR'].append(alg.angles[key])
        result_dict['predicted_opt_precision'].append(alg.precisions[key])
image_container=np.zeros( (1, 2*X+1, 2*X+1), dtype=np.float64)
image_names=[]
# charge the image
image_container[0]=image.astype(np.float64)
image_names.append(f"{fc.selected_filename}")

# charge the image loader:
image_loader.import_converted_images_as_array(image_container, image_names)
# Execute the Rotation and Mirror Algorithms:
# ROTATION ######
interpolation_flag=None
# the interpolation algorithm used in case we disbale its usage for the iX image obtention will be the Lanczos one
rotation_algorithm.interpolation_flag=interpolation_flag if interpolation_flag is not None else cv2.INTER_CUBIC
rotation_algorithm.reInitialize(image_loader)
rotation_algorithm.quadratic_fit_search(precision_quadratic, max_it_quadratic, cost_tolerance_quadratic)
to_result_dict( individual_image_results, rotation_algorithm, "Rotation", "Quadratic", image_names)
rotation_algorithm.reInitialize(image_loader)
rotation_algorithm.fibonacci_ratio_search(precision_fibonacci, max_points_fibonacci, cost_tolerance_fibonacci)
to_result_dict( individual_image_results, rotation_algorithm, "Rotation", "Fibonacci", image_names)

# MIRROR #######
mirror_algorithm.interpolation_flag=interpolation_flag if interpolation_flag is not None else cv2.INTER_CUBIC
mirror_algorithm.reInitialize(image_loader)
mirror_algorithm.quadratic_fit_search(precision_quadratic, max_it_quadratic, cost_tolerance_quadratic)
to_result_dict( individual_image_results, rotation_algorithm, "Mirror", "Quadratic", image_names)
mirror_algorithm.reInitialize(image_loader)
mirror_algorithm.fibonacci_ratio_search(precision_fibonacci, max_points_fibonacci, cost_tolerance_fibonacci)
to_result_dict( individual_image_results, rotation_algorithm, "Mirror", "Fibonacci", image_names)

# GRADIENT #######
def compute_intensity_gravity_center(image):
    """
        Expects input image to be an array of dimensions [h, w].
        It will return an array of gravity centers [2(h,w)] in pixel coordinates
        Remember that pixel coordinates are set equal to numpy indices

    """
    # image wise total intensity and marginalized inensities for weighted sum
    intensity_in_w = np.sum(image, axis=0) # weights for x [raw_width]
    intensity_in_h = np.sum(image, axis=1) # weights for y [raw_height]
    total_intensity = intensity_in_h.sum()

    # Compute mass center for intensity
    # [2] (h_center,w_center)
    return np.nan_to_num( np.stack(
        (np.dot(intensity_in_h, np.arange(image.shape[0]))/total_intensity,
         np.dot(intensity_in_w, np.arange(image.shape[1]))/total_intensity)
        ) )

optimal_masked_gravs={}
optimal_radii={}
grav=compute_intensity_gravity_center(image)

gradient_algorithm.interpolation_flag=interpolation_flag if interpolation_flag is not None else cv2.INTER_CUBIC
gradient_algorithm.reInitialize(image_loader)
gradient_algorithm.quadratic_fit_search(precision_quadratic, max_it_quadratic, cost_tolerance_quadratic)
to_result_dict( individual_image_results, gradient_algorithm, "Gradient", "Quadratic", image_names)
#optimal_masked_gravs['quad'] = gradient_algorithm.masked_gravs[f"Quadratic_Search_{fc.selected_filename}"]
#optimal_radii['quad'] = gradient_algorithm.optimals[f"Quadratic_Search_{fc.selected_filename}"]

gradient_algorithm.reInitialize(image_loader)
gradient_algorithm.fibonacci_ratio_search(precision_fibonacci, max_points_fibonacci, cost_tolerance_fibonacci)
to_result_dict( individual_image_results, gradient_algorithm, "Gradient", "Fibonacci", image_names)

#optimal_masked_gravs['fibo'] = gradient_algorithm.masked_gravs[f"Fibonacci_Search_{fc.selected_filename}"]
#optimal_radii['fibo'] = gradient_algorithm.optimals[f"Fibonacci_Search_{fc.selected_filename}"]

#masked_grav=(optimal_masked_gravs['quad']+optimal_masked_gravs['fibo'])/2.0
#optimal_radi = (optimal_radii['quad']+optimal_radii['fibo'])/2
#print(f"\n\nOptimal masked gravs: {optimal_masked_gravs}\nOptimal radii: {optimal_radii}\n\n\n")
print(pd.DataFrame.from_dict(individual_image_results))

# 7. PROCESS FINAL RESULTS ##########################################
def angle_to_pi_pi( angle): # convert any angle to range ()-pi,pi]
    angle= angle%(2*np.pi) # take it to [-2pi, 2pi]
    return angle-np.sign(angle)*2*np.pi if abs(angle)>np.pi else angle    

average_found_phiCR=np.mean([angle_to_pi_pi(phi) for i,phi in enumerate(individual_image_results['found_phiCR']) if individual_image_results['polarization_method'][i]!='Gradient'])
print("Average found phiCR:", average_found_phiCR)
#print(f"\n\nPredicted slope for main axis: by Gradient {(masked_grav[0]-grav[0])/(masked_grav[1]-grav[1])} and by the others averaged {np.tan(-average_found_phiCR)}")

In [ ]:
'''
def forward(self, x): # [batch_size, 2X+1, 2X+1] or [batch_size, 1, 2X+1, 2X+1]
    x = x.view(x.shape[0], 1, x.shape[-2], x.shape[-1]) # [batch_size, 1, 2X+1, 2X+1]
    X=302
    feats_1=15
    feats_2=20
    feats_3=20
    feats_4=20
    prop1=3
    prop2=2
    prop3=1
    av_pool1_div=4
    conv4_feat_size=15
    av_pool2_div=10
    out_fc_1=10 
    print(x.shape, 2*X+1)

    x = self.relu( self.conv1(x) ) # [batch_size, feats_1, prop1*(2X+1)/5, prop1*(2X+1)/5]
    print("conv1",x.shape, prop1*(2*X+1)/5)


    x = self.batchNorm2( self.relu( self.conv2(self.dropout1(x)) )) # [batch_size, feats_2, prop2*(2X+1)/5, prop2*(2X+1)/5]
    print("conv2",x.shape,  prop2*(2*X+1)/5)


    x = self.relu( self.conv3(self.dropout2(x)) ) # [batch_size, feats_3, prop3*(2X+1)/5, prop3*(2X+1)/5]
    print("conv3",x.shape,  prop3*(2*X+1)/5)


    x = self.avPool1(x) # [batch_size, feats_3, prop3*(2X+1)/5)/av_pool1_div, prop3*(2X+1)/5)/av_pool1_div]
    print("av_pool1",x.shape, int((prop3*(2*X+1)/5)/av_pool1_div))


    x = self.batchNorm4(self.conv4(self.dropout2(x))) # [batch_size, feats_4, conv4_feat_size, conv4_feat_size]
    print("conv4+batchn",x.shape, conv4_feat_size)


    x = self.relu( self.avPool2(x) ) # [batch_size, feats_4, conv4_feat_size/av_pool2_div, conv4_feat_size/av_pool2_div]
    print("av_pool2",x.shape, int(conv4_feat_size/av_pool2_div)+1)


    x = x.view(x.shape[0], self.in_fc) #[batch_size, feats_4*int(conv4_feat_size/av_pool2_div)**2]
    print("view_change",x.shape, feats_4*int(conv4_feat_size/av_pool2_div+1)**2)


    x = self.fc2( self.relu( self.fc1(x) ) ) #[batch_size, 4]
    print(x.shape, 4)

        return x
a = Simple_Encoder().to(device)
a(torch.ones(2,1, 605,605).to(device))
del a
torch.cuda.empty_cache()
'''

In [ ]:
def train_crazy_epoch(epoch, criterion, model, optimizer, datas, targets, batch_number, batch_size,
                      print_loss_every_batches=20,
                    optimizer_step_every_batches=1):
    
    total_loss = 0.0

    model.train()

    optimizer.zero_grad()
    t2 = time()
    for k in range(batch_number):        
        
        prediction = model(datas[k*batch_size:(k+1)*batch_size]) # data is [batch_size, 1, 2X+1, 2X+1]
        loss = criterion(prediction, targets[k*batch_size:(k+1)*batch_size])
        loss.backward()
        
        if k % optimizer_step_every_batches==optimizer_step_every_batches-1:
            optimizer.step()
            optimizer.zero_grad()
        # print loss every N batches
        if k % print_loss_every_batches == print_loss_every_batches-1:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (k+1) * batch_size, len(datas),
                100*(k+1)*batch_size / len(datas), loss.item()))

        #total_loss += loss.item()  #.item() is very important here
        # Why?-> In order to avoid having total_loss as a tensor in the gpu
        t1= time()
        print(f"Iteration time{t1-t2}")
        t2 = time()

    return total_loss / len(datas)

In [ ]:
def full_crazy_training_loop(model, criterion, optimizer_generator, train_loader,
                             batch_number, batch_size, epochs=10,
                       print_loss_every_batches=20, optimizer_step_every_batches=1,
                            meta_epoch_number=1):
    %matplotlib inline
    for meta_epoch in range(meta_epoch_number):
        for meta_batch_id, (datas, targets) in enumerate(train_loader):        
            datas, targets = datas.to(device), targets.to(device) # pero muuh gordos
            losses = {"train": []}
            optimizer = optimizer_generator(model)
            for epoch in range(epochs): # que overfitee el muuh gordo este
                train_loss = train_crazy_epoch(epoch, criterion, model, optimizer, datas,
                                         targets, batch_number, batch_size,
                                          print_loss_every_batches=20,
                                            optimizer_step_every_batches=1)

                display_IPython.clear_output(wait=True)
                losses["train"].append(train_loss)
                plt.plot(losses["train"], label=f"log training loss- MetaBatch {meta_batch_id/len(train_loader)*100}%")
                plt.yscale('log')
                plt.legend()
                plt.pause(0.001)
                plt.show()   
    return losses

In [ ]:
meta_epoch_number = 1
meta_batch_size = 100
batch_size = 10
batch_number = int(meta_batch_size/batch_size)
assert(meta_batch_size%batch_size==0)

crazy_loader = DataLoader(training_data, batch_size=meta_batch_size, shuffle=True, num_workers=worker_num,
                              pin_memory=True, drop_last=False, persistent_workers=False)

def adam_generator(model):
    return torch.optim.Adam(model.parameters(), lr=0.05, betas=(0.99, 0.9999), eps=1e-08, weight_decay=0, amsgrad=False)

full_crazy_training_loop(model, criterion, 
                         adam_generator, 
                         crazy_loader,
                             batch_number, batch_size, epochs=10,
                       print_loss_every_batches=10, optimizer_step_every_batches=2, 
                         meta_epoch_number=meta_epoch_number)

In [ ]:
t1=time()
for datas, targets in train_dataloader:
    datas, targets = datas.to(device), targets.to(device)
    pred = model(datas)
    t2=time()
    print(f"inf time {t2-t1}")
    loss = criterion(pred, targets)
    loss.backward()
    t3=time()
    print(f"with backward {t3-t1}")
    t1=time()